In [6]:
import re
import time
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV,ElasticNet, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
import matplotlib.pyplot as plt

test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

### EDA 

In [7]:
x = train.drop(columns = "price")
y = train['price'] 

# Get numeric variables :)
x_num = x.select_dtypes(include = 'number')

# Get rid of bedrooms cause it's a dumb variable
x_num = x_num.drop(columns = 'bedrooms')

# Got just the unit id
ids = train['Id']

# Split training data into train/test :)
x_train, x_test, y_train, y_test = train_test_split(x_num, y, test_size= 0.2, random_state=317)

#### PIPELINES FOR THE WIN

In [28]:
# Build Pipeline
pipe = Pipeline([
    ('impute', SimpleImputer(strategy= 'mean')),
    ('scale', StandardScaler()),
    ('en', ElasticNetCV(l1_ratio=.75))
])

# Fit pipeline to training data
pipe.fit(x_train, y_train)

# Report test MSE
y_preds = pipe.predict(x_test)
root_mean_squared_error(y_test, y_preds)

1261.3373567780138

In [34]:
# Fit KNN regression with k = 10
knn = KNeighborsRegressor(n_neighbors= 30)
pipe2 = Pipeline([
    ('impute', SimpleImputer(strategy= 'mean')),
    ('scale', StandardScaler()),
    ('model', knn)
])

# Fit pipeline to training data
pipe2.fit(x_train, y_train)

# Report test MSE
y_preds2 = pipe2.predict(x_test)
root_mean_squared_error(y_test, y_preds2)

1256.7135790682428

### Actual Test Set :)